In [8]:
# import hydra
# from hydra import initialize, compose
# from omegaconf import OmegaConf
# import torch
# import os

# # 1. Hydra 초기화
# hydra.core.global_hydra.GlobalHydra.instance().clear()

# # 2. Config 로드
# with initialize(version_base=None, config_path="configs"):
#     cfg = compose(config_name="test_CHJ.yaml")

# print("Original Config Structure Loaded.")

# # ==============================================================================
# # [핵심 해결책] 중첩된 설정(Nested Config)을 평평하게 펴주기 (Flattening)
# # ==============================================================================
# # 모델 코드가 args.gnn_hidden_dim, args.num_beams 처럼 바로 접근하기 때문에
# # 'trainer'와 'gnn' 안에 있는 설정들을 최상위(root)로 꺼내와 합쳐야 합니다.

# OmegaConf.set_struct(cfg, False)  # 구조 변경 허용 (잠금 해제)

# # 1) Trainer 설정 꺼내기 (num_beams, gen_max_len 등 해결)
# if "trainer" in cfg:
#     print("Merging 'trainer' config into root...")
#     cfg = OmegaConf.merge(cfg, cfg.trainer)

# # 2) GNN 설정 꺼내기 (gnn_hidden_dim, gine, tokengt 등 해결)
# # ★ 이 부분이 추가되어야 'gnn_hidden_dim' 에러가 사라집니다.
# if "gnn" in cfg:
#     print("Merging 'gnn' config into root...")
#     cfg = OmegaConf.merge(cfg, cfg.gnn)

# # ==============================================================================
# # [경로 수정] GINE Checkpoint 경로 재설정
# # ==============================================================================
# # 병합(Merge) 후에는 'gine' 설정이 최상위에 위치하게 됩니다 (cfg.gine).
# # 따라서 cfg.gine.graph_encoder_ckpt 를 수정해야 합니다.

# # 다시 구조 잠금 (안전장치)
# OmegaConf.set_struct(cfg, True)

# # ==============================================================================
# # [검증] 주요 키들이 최상위에 잘 나왔는지 확인
# # ==============================================================================
# print("-" * 40)
# print(f"1. num_beams (from trainer): {cfg.get('num_beams', 'MISSING')}") 
# print(f"2. gnn_hidden_dim (from gnn): {cfg.get('gnn_hidden_dim', 'MISSING')}")
# print(f"3. GINE Path: {cfg.gine.graph_encoder_ckpt if hasattr(cfg, 'gine') else 'MISSING'}")
# print("-" * 40)

# # ==============================================================================
# # 모델 로드
# # ==============================================================================
# ckpt_path = "/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt"
# correct_token_path = "/home/jovyan/CHJ/Mol-LLM_Custom/model/selfies_dict.txt"


In [9]:
# cfg_dict = {
#     "data": {
#         "raw_data_root": "/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train",
#         "data_tag": "3.3M_0415",
#         "tasks": ["reagent_prediction"],
#     },
#     "gnn": {
#         "gnn_type": "gine_tokengt",
#         "gnn_hidden_dim": 1024,
#         "gine": {
#             "gnn_hidden_dim": 1024,
#             "gin_num_layers": 5,
#             "drop_ratio": 0.0,
#             "used_gnn_layer": -1,
#             "gnn_jk": "last",
#             "graph_encoder_ckpt": "/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt",
#             "gnn_type": "gine",
#         },
#         "tokengt": {
#             "input_feat_dim": 9,
#             "gnn_hidden_dim": 1024,
#             "num_layers": 5,
#             "num_heads": 8,
#             "method": "laplacian",
#             "d_p": 64,
#             "d_e": 64,
#             "use_graph_token": True,
#             "max_position_embeddings": 102,
#             "graph_encoder_ckpt": "/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt",
#             "gnn_type": "tokengt",
#         },
#     },
#     "trainer": {
#         "bert_hidden_dim": 768,
#         "bert_name": "scibert",
#         "cross_attention_freq": 2,
#         "num_query_token": 32,
#         "bert_num_hidden_layers": 5,
#         "projector_type": "qformer",
#         "llm_model": "mistralai/Mistral-7B-Instruct-v0.3",
#         "tune_llm": "lora",
#         "peft_config": None,
#         "peft_dir": "",
#         "load_in_8bit": False,
#         "lora_r": 64,
#         "lora_alpha": 32,
#         "lora_dropout": 0.1,
#         "selfies_token_path": "Mol-LLM_Custom/model/selfies_dict.txt",
#         "add_selfies_tokens": True,
#         "prompt": "[START_I_SMILES]{}[END_I_SMILES]",
#         "num_beams": 1,
#         "strategy_name": None,
#         "accelerator": "gpu",
#         "devices": "0,1,2,3,4,5,6,7",
#         "precision": "bf16-mixed",
#         "max_steps": -1,
#         "max_epochs": 12,
#         "every_n_epochs": 1,
#         "task": None,
#         "logging_dir": "/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom",
#         "llava_pretraining": 0,
#         "second_stage_start_epoch": 4,
#         "num_workers": 0,
#         "skip_sanity_check": False,
#         "total_batch_size": 512,
#         "batch_size": 8,
#         "inference_batch_size": 11,
#         "truncation": 1,
#         "padding": "max_length",
#         "max_length": 512,
#         "inference_max_length": 512,
#         "gen_max_len": 256,
#         "min_len": 8,
#         "apply_sequence_packing": False,
#         "max_packing_size": -1,
#         "weight_decay": 0.05,
#         "min_lr": 1e-05,
#         "init_lr": 0.0001,
#         "warmup_lr": 1e-05,
#         "warmup_epochs": 0.25,
#         "scheduler": "linear_warmup_cosine_lr",
#         "optimizer": "adamw",
#         "log_every_n_steps": 50,
#         "gradient_clip_val": 0.5,
#         "val_check_interval": 0.5,
#         "test_on_trainset": False,
#         "mol_representation": "string+graph",
#         "log_attn_score": True,
#         "eval_modality_util": None,
#         "tune_gnn": True,
#         "train_molpo": False,
#         "eval_molpo": False,
#         "find_unused_parameters": False,
#         "selfies_enumeration": False,
#         "isomericSmiles": False,
#         "canonical": False,
#         "allHsExplicit": False,
#     },
#     "filename": "debugging",
#     "seed": 42,
#     "mode": "ft",
#     "wandb_entity": "hj_ai",
#     "wandb_project": "mol-llm",
#     "wandb_log_freq": 100,
#     "wandb_id": None,
#     "debug": False,
#     "ckpt_path": None,
#     "pretrained_ckpt_path": None,
#     "shuffle_selfies": False,
#     "shuffle_graph": False,
#     "process_disjoint": True,
# }

In [12]:
import torch
import hydra
from hydra import initialize, compose
from omegaconf import OmegaConf
import os
import selfies as sf
from ogb.utils import smiles2graph
from torch_geometric.data import Data, Batch
from model.blip2_stage3 import Blip2Stage3
import logging

# ==============================================================================
# 0. 모델 로드 및 수술 함수 (Weight Tying & Repair)
# ==============================================================================
def load_and_fix_model(model, ckpt_path):
    print(f"\n[Repair Loader] Loading weights from {ckpt_path}...")
    checkpoint = torch.load(ckpt_path, map_location="cpu")
    state_dict = checkpoint["state_dict"] if "state_dict" in checkpoint else checkpoint
    
    # 1. 가중치 로드 (기존 방식)
    model_state_dict = model.state_dict()
    new_state_dict = {}
    
    # 키 매핑 및 필터링
    for k, v in state_dict.items():
        # 접두사 처리 (blip2model. 등)
        target_key = k
        if k not in model_state_dict:
            if k.startswith("blip2model.") and k[11:] in model_state_dict:
                target_key = k[11:]
            elif "blip2model." + k in model_state_dict:
                target_key = "blip2model." + k
        
        if target_key in model_state_dict:
            new_state_dict[target_key] = v

    # 로드 실행 (Base Model의 Frozen Weight는 무시)
    missing, _ = model.load_state_dict(new_state_dict, strict=False)
    print("[Repair Loader] Checkpoint weights loaded.")

    # 2. [핵심] LM Head 수술 (Weight Tying)
    # LoRA 모델 구조상 llm_model 내부 깊숙한 곳에 접근해야 함
    try:
        # 경로 찾기: model.blip2model.llm_model ...
        llm = model.blip2model.llm_model
        
        # PeftModel로 감싸져 있는 경우 base_model 안으로 진입
        if hasattr(llm, "base_model"):
            base = llm.base_model.model 
        else:
            base = llm.model

        # Embed Tokens (입력) 와 LM Head (출력) 가져오기
        # 구조: MistralForCausalLM -> model(MistralModel) -> embed_tokens
        # 구조: MistralForCausalLM -> lm_head
        
        # 정확한 모듈 가져오기
        if hasattr(base, "model") and hasattr(base.model, "embed_tokens"):
            embed_tokens = base.model.embed_tokens
        elif hasattr(base, "embed_tokens"):
            embed_tokens = base.embed_tokens
        else:
            print("⚠️ Cannot find embed_tokens module.")
            embed_tokens = None

        if hasattr(base, "lm_head"):
            lm_head = base.lm_head
        elif hasattr(llm, "lm_head"): # PeftModel wrapper 바깥에 있을 수도 있음
            lm_head = llm.lm_head
        else:
            print("⚠️ Cannot find lm_head module.")
            lm_head = None

        # 가중치 복사 (Tying)
        if embed_tokens is not None and lm_head is not None:
            print(f"   Input Embeddings Shape: {embed_tokens.weight.shape}")
            print(f"   Output LM Head Shape  : {lm_head.weight.shape}")
            
            # 크기가 같다면 가중치 복사 (학습된 입력 임베딩을 출력에도 적용)
            if embed_tokens.weight.shape == lm_head.weight.shape:
                print("🔧 [FIX] Copying Embeddings to LM Head (Fixing Garbage Output)...")
                # Clone하여 복사 (메모리 공유가 아니라 값 복사)
                lm_head.weight.data = embed_tokens.weight.data.clone()
                print("✅ [SUCCESS] LM Head fixed!")
            else:
                print("⚠️ Shape mismatch! Cannot tie weights.")
        else:
            print("⚠️ Modules not found for tying.")

    except Exception as e:
        print(f"⚠️ Error during model repair: {e}")


# ==============================================================================
# 1. Config & Path Setup
# ==============================================================================
hydra.core.global_hydra.GlobalHydra.instance().clear()

with initialize(version_base=None, config_path="configs"):
    cfg = compose(config_name="test_CHJ.yaml")

OmegaConf.set_struct(cfg, False)
if "trainer" in cfg: cfg = OmegaConf.merge(cfg, cfg.trainer)
if "gnn" in cfg: cfg = OmegaConf.merge(cfg, cfg.gnn)

# 경로 설정
root_dir = "/home/jovyan/CHJ/Mol-LLM_Custom"
main_ckpt_path = os.path.join(root_dir, "checkpoint/Custom/mol-llm.ckpt")
token_path = os.path.join(root_dir, "model/selfies_dict.txt")

if hasattr(cfg, "gine"): cfg.gine.graph_encoder_ckpt = main_ckpt_path
if hasattr(cfg, "tokengt"): cfg.tokengt.graph_encoder_ckpt = main_ckpt_path
cfg.selfies_token_path = token_path
OmegaConf.set_struct(cfg, True)

# ==============================================================================
# 2. 모델 초기화 및 로드
# ==============================================================================
print(f"Initializing Model...")
model = Blip2Stage3(cfg) 

# [수정된 로더 호출]
load_and_fix_model(model, main_ckpt_path)

model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Moving model to {device} and converting to bfloat16...")
model.to(device)
model.to(torch.bfloat16)

# ==============================================================================
# 3. 데이터 준비
# ==============================================================================
tokenizer = model.blip2model.llm_tokenizer
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

mol_token_id = tokenizer.mol_token_id
mol_token = "<mol>"
num_query_tokens = 32

# 입력 데이터
input_selfies = "[C][O][C@H1][C][Branch1][C][C][=C][C@H1][C][C@H1][Ring1][Ring1][Ring1][#Branch1]"
instruction_template = "Please provide the HOMO energy value for this molecule: : <INPUT> "
system_prompt = "You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation."

"""
<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nPlease provide the HOMO energy value for this molecule: <SELFIES> [C][O][C@H1][C][Branch1][C][C][=C][C@H1][C][C@H1][Ring1][Ring1][Ring1][#Branch1] </SELFIES><GRAPH><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol><mol></GRAPH>. [/INST] 
"""

def smiles2data(smiles):
    try:
        graph = smiles2graph(smiles)
        x = torch.from_numpy(graph["node_feat"]).long()
        edge_index = torch.from_numpy(graph["edge_index"]).long()
        edge_attr = torch.from_numpy(graph["edge_feat"]).long()
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    except Exception:
        return None

smiles = sf.decoder(input_selfies)
graph = smiles2data(smiles)
graph_batch = Batch.from_data_list([graph]).to(device)
additional_graph_batch = Batch.from_data_list([graph]).to(device)

formatted_selfies = f"<SELFIES> {input_selfies} </SELFIES>"
graph_placeholder = "<GRAPH>" + mol_token * num_query_tokens + "</GRAPH>"
input_mol_string = formatted_selfies + graph_placeholder
final_instruction = instruction_template.replace("<INPUT>", input_mol_string)
full_prompt = f"<s>[INST] {system_prompt} \n\n{final_instruction} [/INST]"

inputs = tokenizer(
    [full_prompt], 
    return_tensors="pt", 
    add_special_tokens=False, 
    padding=True
)
input_ids = inputs.input_ids.to(device)
attention_mask = inputs.attention_mask.to(device)
is_mol_token = (input_ids == mol_token_id)

# ==============================================================================
# 4. 추론 실행
# ==============================================================================
print("\nGenerating response...")

with torch.no_grad():
    with torch.amp.autocast(device_type="cuda", dtype=torch.bfloat16):
        outputs = model.blip2model.generate(
            graphs=(graph_batch, additional_graph_batch),
            input_ids=input_ids,
            attention_mask=attention_mask,
            is_mol_token=is_mol_token,
            num_beams=5,             
            max_length=256, 
            min_length=1,
            do_sample=False, 
            repetition_penalty=1.0
        )

# ==============================================================================
# 5. 결과 출력
# ==============================================================================
raw_prediction = outputs.predictions[0]
clean_prediction = raw_prediction.replace(tokenizer.pad_token, "").replace("</s>", "").strip()

print("\n" + "="*50)
print("Prediction Result:")
print(clean_prediction)
print("="*50)

/home/jovyan/miniconda3/envs/MolDA_CHJ/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'test_CHJ.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


Initializing Model...
{'data': {'raw_data_root': '/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train', 'data_tag': '3.3M_0415', 'tasks': ['reagent_prediction']}, 'gnn': {'gnn_type': 'gine_tokengt', 'gnn_hidden_dim': 1024, 'gine': {'gnn_hidden_dim': 1024, 'gin_num_layers': 5, 'drop_ratio': 0.0, 'used_gnn_layer': -1, 'gnn_jk': 'last', 'graph_encoder_ckpt': '/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt', 'gnn_type': 'gine'}, 'tokengt': {'input_feat_dim': 9, 'gnn_hidden_dim': 1024, 'num_layers': 5, 'num_heads': 8, 'method': 'laplacian', 'd_p': 64, 'd_e': 64, 'use_graph_token': True, 'max_position_embeddings': 102, 'graph_encoder_ckpt': '/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt', 'gnn_type': 'tokengt'}}, 'trainer': {'bert_hidden_dim': 768, 'bert_name': 'scibert', 'cross_attention_freq': 2, 'num_query_token': 32, 'bert_num_hidden_layers': 5, 'projector_type': 'qformer', 'llm_model': 'mistralai/Mistral-7B-Instruct-v0.3', 'tune_llm': 'lora', 'peft_config

loading file tokenizer.model from cache at /home/jovyan/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/c170c708c41dac9275d15a8fff4eca08d52bab71/tokenizer.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jovyan/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/c170c708c41dac9275d15a8fff4eca08d52bab71/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jovyan/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/c170c708c41dac9275d15a8fff4eca08d52bab71/tokenizer_config.json
loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/c170c708c41dac9275d15a8fff4eca08d52bab71/tokenizer.json


Added 2944 selfies tokens to the tokenizer


loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/c170c708c41dac9275d15a8fff4eca08d52bab71/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 32768
}

loading weights file model.safetensors from cache at /home/jovyan/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/c170c708c41dac9275

trainable params: 167,772,160 || all params: 7,440,183,296 || trainable%: 2.2549
/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt -args.gine.graph_encoder_ckpt
load graph encoder from /home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt
load graph encoder from /home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt
bert load scibert


loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading weights file pytorch_model.bin from cache at /home/jovyan/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/pytorch_model.bin
Generate config GenerationConfig {
  "pad_token_id": 0
}

Some weigh


[Repair Loader] Loading weights from /home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/Custom/mol-llm.ckpt...
[Repair Loader] Checkpoint weights loaded.
   Input Embeddings Shape: torch.Size([35745, 4096])
   Output LM Head Shape  : torch.Size([35745, 4096])
🔧 [FIX] Copying Embeddings to LM Head (Fixing Garbage Output)...
✅ [SUCCESS] LM Head fixed!
Moving model to cuda and converting to bfloat16...


/home/jovyan/miniconda3/envs/MolDA_CHJ/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generating response...

Prediction Result:
[C@@H1] [125Te] [6Li] [33ClH1] [\14C@H1] [\14C@H1] [Rf] [Rf] [Rf] [82Se] [/ClH1+1] [82Se] [82Se] [=La] [115Cd] [246Pu] [38K] [146Sm] [/ClH1+1] [Tc] [11B-1] [11B-1] [230U] [47V] [82Se] [194Os] [194Os] [145Pm] [123I-1] [233U+4] [233U+4] [82Se] [82Se] [/S@] [TlH1] [N@@] [N@@] [154Sm] [N@@] [6Li+1] [Sn@@] [171Yb] [=FeH1] [\PH1+1] [171Tm] [89Rb] [89Rb] [154Sm] [79Rb] [82Se] [Ho] [Cr-1] [194Os] [110Sn] [236U] [152Gd] [152Gd] [152Gd] [152Gd] [152Gd] [152Gd] [=Fe] [137Xe] [137Xe] [=Fe] [246Pu] [=Fe] [65Zn] [194Os] [Te+1] [Te+1] [136Xe] [33ClH1] [33ClH1] [33ClH1] [=Fe] [81Rb] [150Eu] [185Ir] [#Ti] [82Se] [/Cl-1] [/Cl-1] [/Cl-1] [230U] [81Rb] [82Se] [82Se] [82Se] [82Se] [82Se] [82Se] [82Se] [82Se] [82Se] [194Os] [54Cr] [45K+1] [230U] [6Li+1] [40PH3] [40PH3] [/ClH1+1] [81Rb] [18CH1] [81Rb] [BH1-1] [#14C] [82Se] [82Se] [Co-3] [6Li+1] [246Pu] [33ClH1] [33ClH1] [127Sb+3] [127Sb+3] [11B-1] [\Sn] [\Sn-1] [82Se] [92Nb] [69Ga+3] [33ClH1] [33ClH1] [33ClH1] [33C

In [11]:
from datasets import load_from_disk

data = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/checkpoint/mol-llm_testset')

data[0]

{'task': 'alchemy_homo',
 'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [7, 0, 2, 5, 0, 0, 2, 0, 0],
  [5, 2, 4, 5, 1, 0, 2, 0, 1],
  [5, 2, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 2, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 3, 5, 1, 0, 1, 0, 1],
  [5, 0, 3, 5, 0, 0, 1, 0, 1],
  [5, 0, 4, 5, 3, 0, 2, 0, 0]],
 'edge_index': [[0, 1, 1, 2, 2, 3, 2, 7, 3, 4, 3, 5, 4, 5, 5, 6, 6, 7, 7, 8],
  [1, 0, 2, 1, 3, 2, 7, 2, 4, 3, 5, 3, 5, 4, 6, 5, 7, 6, 8, 7]],
 'edge_attr': [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [1, 0, 0],
  [1, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 'additional_x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [7, 0, 2, 5, 0, 0, 2, 0, 0],
  [5, 2, 4, 5, 1, 0, 2, 0, 1],
  [5, 2, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 2, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 3, 5, 1, 0, 1, 0, 1],
  [5, 0, 3, 5, 0, 0, 1, 0, 1],
  [5, 0, 